<a href="https://colab.research.google.com/github/ckinzthompson/ckinzthompson.github.io/blob/main/additional/MS_MLE_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate Mass and Charge from Peaks (MLE)

In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display,clear_output

def create_input(ind):
	iw = widgets.IntText(
		value = ind,
		description='',
		disabled=False,
	)
	fw = widgets.FloatText(
		value=None,
		description='',
		disabled=False,
	)
	return [iw,fw]

def add_input(b):
	n_cells = len(data_widgets)-1
	maximum_user = np.max([dw[0].value for dw in data_widgets])
	data_widgets.append(create_input(1+np.max([n_cells,maximum_user])))
	update_widgets()

def remove_input(b):
	if data_widgets:
		data_widgets.pop()
		update_widgets()

def update_widgets():
	box.children = [
		widgets.HBox([widgets.Label('Rows: '),add_button,remove_button]),
		widgets.Label(r'Relative Charge,   Peak(m/z)'),
		widgets.VBox([widgets.HBox(dw) for dw in data_widgets]),
		
		calculate_button,
		output,
	]

def calculate(b):
	_charges = np.array([dw[0].value for dw in data_widgets]) 
	_peaks = np.array([dw[1].value for dw in data_widgets])
	keep = _peaks != 0.0
	charges = _charges[keep].copy()
	peaks = _peaks[keep].copy()

	if np.any(peaks[1:]>peaks[:-1]):
		order = np.argsort(peaks)[::-1]
		peaks = peaks[order]
		charges = charges[order]

	if np.all(peaks[1:]>peaks[:-1]):
		peaks = peaks[::-1]
		charges = charges[::-1]

	if not np.all(peaks.argsort() == charges.argsort()):
		order = np.argsort(peaks)[::-1]
		peaks = peaks[order]
		charges = charges[order]

	## check for polymer
	if peaks.size > 2:
		dx = peaks[:-1]-peaks[1:]
		if dx.std() < 1.:
			with output:
				print('Avg. Delta: %.4f, Std. Dev. Delta: %.4f'%(dx.mean(),dx.std()))
				print('Some sort of Polymer?')
			return

	## estimate charges
	x0 = peaks[:-1]
	x1 = peaks[1:]
	m = x0*x1/(x0-x1)
	m = np.median(m)
	z = m/(peaks-1.)

	## find outliers and remove them
	keep = np.abs(z-(z+.5)//1) < 0.05
	if keep.sum() <= 2:
		keep = np.ones(z.size,dtype='bool')

	x = peaks[keep]
	n = charges[keep]

	## MLE calculate mass and base charge
	E_x = np.mean(x)
	E_x2 = np.mean(x**2.)
	E_xm1_n = np.mean((x-1.00784)*n)
	E_x_xm1_n = np.mean(x*(x-1.00784)*n)

	z = (E_x * E_xm1_n - E_x_xm1_n)/(E_x2 - E_x**2.)
	m = z*E_x + E_xm1_n
	
	## output
	with output:
		clear_output(wait=True)
		print("Mass: %.3f"%(m))
		print('Delta Charge: %.2f'%(z))

		print('----------------')
		print('z, Observed (m/z), z, Predicted (m/z), Delta(m/z)')
		for i in range(peaks.size):
			zi = int(np.round(z+_charges[i],0))
			print('%d:\t%.4f,\t%.4f,\t%.4f'%(zi, peaks[i], (m+charges[i])/zi, np.abs(peaks[i]- (m+charges[i])/zi)))

data_widgets = [create_input(i) for i in range(10)]

add_button = widgets.Button(description='+')
add_button.on_click(add_input)

remove_button = widgets.Button(description='-')
remove_button.on_click(remove_input)

calculate_button = widgets.Button(description='Calculate')
calculate_button.on_click(calculate)

output = widgets.Output()

box = widgets.VBox([
	widgets.HBox([widgets.Label('Rows: '),add_button,remove_button]),
	widgets.Label(r'Relative Charge,   Peak(m/z)'),
	widgets.VBox([widgets.HBox(dw) for dw in data_widgets]),
	
	calculate_button,
	output,
])

display(box)

# Calculate Peaks from Mass

In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display,clear_output

mass_input = widgets.FloatText(description="Mass (Da):",value=30000.0001)
maxcharge_input = widgets.IntText(description="Max Charge:",value=50)
button = widgets.Button(description="Calculate")
output2 = widgets.Output()

def calculate(b):
	mass = mass_input.value
	maxcharge = maxcharge_input.value
	with output2:
		clear_output(wait=True)
		print('0:\t%.4f'%(mass))
		for i in range(1,maxcharge+1):
			print('+%d:\t%.4f'%(i,(mass+i*1.00784)/(i*1.00784)))
	
button.on_click(calculate)
display(mass_input,maxcharge_input,button,output2)

